In [1]:
import torch
from pykeops.torch import LazyTensor
from gpytorch.kernels.keops import RBFKernel
from gpytorch.lazy import LazyEvaluatedKernelTensor

In [2]:
torch.set_default_dtype(torch.float64)

In [3]:
n = 10000
d = 3
sigma = 1.0
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

A = torch.randn(n, d, device=device)
B = torch.randn(n, n, device=device)

PyKeops -- doesn't work

In [4]:
# Ai_lazy = LazyTensor(A[:, None, :])  # (100, 1, 3)
# Aj_lazy = LazyTensor(A[None, :, :])  # (1, 100, 3)
# # Compute the squared Euclidean distance matrix
# D_lazy = ((Ai_lazy - Aj_lazy) ** 2).sum(dim=2)  # (100, 100)
# D_lazy - B

GPyTorch

In [5]:
kernel = RBFKernel()
kernel.lengthscale = torch.tensor(sigma)
kernel = kernel.to(device)
# print(kernel.lengthscale)
# print(kernel.lengthscale.device)
K_lazy = kernel(A, A)

In [6]:
# Compute the actual kernel matrix via pytorch
K_true = torch.exp(-((A[:, None, :] - A[None, :, :])
                   ** 2).sum(dim=2)/(2 * sigma ** 2))

In [7]:
# torch.sum(K_lazy ** 2)

In [8]:
print(torch.allclose(K_lazy @ B, K_true @ B))
print(torch.allclose(B @ K_lazy, B @ K_true))

True
True


In [9]:
print(torch.allclose((K_lazy - B) @ B, (K_true - B) @ B))
print(torch.allclose((K_lazy - B).T @ B, (K_true - B).T @ B))
print(torch.allclose(B @ (K_lazy - B), B @ (K_true - B)))

True
True
True


In [10]:
print(torch.allclose(B[:, :100] @ (K_lazy - B)
      [:100], B[:, :100] @ (K_true - B)[:100]))

True


In [14]:
K_lazy.device

device(type='cuda', index=0)

In [19]:
K_lazy.to_dense()

tensor([[1.0000, 0.6417, 0.2164,  ..., 0.0392, 0.0030, 0.0097],
        [0.6417, 1.0000, 0.0666,  ..., 0.0653, 0.0015, 0.0010],
        [0.2164, 0.0666, 1.0000,  ..., 0.1431, 0.1571, 0.3796],
        ...,
        [0.0392, 0.0653, 0.1431,  ..., 1.0000, 0.1947, 0.0143],
        [0.0030, 0.0015, 0.1571,  ..., 0.1947, 1.0000, 0.1330],
        [0.0097, 0.0010, 0.3796,  ..., 0.0143, 0.1330, 1.0000]],
       device='cuda:0', grad_fn=<MatmulBackward>)

In [20]:
(K_lazy - B).to_dense()

tensor([[ 2.1224, -0.4361,  1.4086,  ..., -0.4366,  0.0819, -0.0093],
        [-0.8981,  1.7875, -0.2049,  ..., -0.5510,  0.6616,  0.9485],
        [ 0.8515, -0.4668,  2.4855,  ..., -0.6536, -0.7341, -0.7959],
        ...,
        [-0.2632, -0.4029, -1.3474,  ...,  0.1396, -0.0564,  1.1475],
        [-0.3088, -0.4416,  0.6186,  ..., -1.9514,  3.3519, -0.1429],
        [ 1.1929, -0.1689, -0.7988,  ...,  0.7487,  1.3666,  1.9589]],
       device='cuda:0', grad_fn=<AddBackward0>)